# Assignment
Scenario:
Company XYZ held a promo sale for their signature items named: x,y,z. Sales are at an
all-time high, but they want to create a marketing strategy to target age groups of people by
looking at total quantities purchased.

# Objectives
Create a Python script that can:
1. connect to the SQLite3 database provided
2. extract the total quantities of each item bought per customer aged 18-35.
- For each customer, get the sum of each item
- Items with no purchase (total quantity=0) should be omitted from the final
list
- No decimal points allowed (The company doesn’t sell half of an item ;) )
Challenge: Provide 2 solutions, one using purely SQL, the other using Pandas
3. store the query to a CSV file, delimiter should be the semicolon character (';')


In [15]:
#1. connect to the SQLite3 database provided


import sqlite3

# Connect to a new SQLite database or an existing one
conn = sqlite3.connect('sales_database.db')
cursor = conn.cursor()


# Creating tables

In [16]:
# Create Customers table
cursor.execute('''
    CREATE TABLE Customers (
        customer_id INTEGER PRIMARY KEY,
        customer_name TEXT NOT NULL,
        age INTEGER NOT NULL
    )
''')

# Create Items table
cursor.execute('''
    CREATE TABLE Items (
        item_id INTEGER PRIMARY KEY,
        item_name TEXT NOT NULL
    )
''')

# Create Sales table
cursor.execute('''
    CREATE TABLE Sales (
        transaction_id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        transaction_date DATE NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
    )
''')

# Create SalesDetails table
cursor.execute('''
    CREATE TABLE SalesDetails (
        sales_detail_id INTEGER PRIMARY KEY,
        transaction_id INTEGER NOT NULL,
        item_id INTEGER NOT NULL,
        quantity INTEGER,
        FOREIGN KEY (transaction_id) REFERENCES Sales(transaction_id),
        FOREIGN KEY (item_id) REFERENCES Items(item_id)
    )
''')


# Inserting Values

In [17]:
# Insert sample data into the Customers table
cursor.executemany('''
    INSERT INTO Customers (customer_name, age) VALUES (?, ?)
''', [
    ('Customer 1', 25),
    ('Customer 2', 30),
    ('Customer 3', 35)
])

# Insert sample data into the Items table
cursor.executemany('''
    INSERT INTO Items (item_name) VALUES (?)
''', [
    ('X'),
    ('Y'),
    ('Z')
])

# Insert sample data into the Sales and SalesDetails tables
cursor.execute('''
    INSERT INTO Sales (customer_id, transaction_date) VALUES (?, ?)
''', (1, '2023-10-01'))

cursor.executemany('''
    INSERT INTO SalesDetails (transaction_id, item_id, quantity) VALUES (?, ?, ?)
''', [
    (1, 1, 10),
    (2, 1, 1),
    (2, 2, 1),
    (3, 3, 2)
])


In [18]:
# Commit the changes and close the database connection
conn.commit()


2. extract the total quantities of each item bought per customer aged 18-35.
- For each customer, get the sum of each item
- Items with no purchase (total quantity=0) should be omitted from the final
list
- No decimal points allowed (The company doesn’t sell half of an item ;) )

# Using SQL

In [19]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('sales_database.db')
cursor = conn.cursor()

# Define and execute the SQL query
query = """
SELECT
    C.customer_id,
    I.item_id,
    SUM(COALESCE(SD.quantity, 0)) AS total_quantity
FROM Customers C
JOIN Sales S ON C.customer_id = S.customer_id
JOIN SalesDetails SD ON S.transaction_id = SD.transaction_id
JOIN Items I ON SD.item_id = I.item_id
WHERE C.age BETWEEN 18 AND 35
GROUP BY C.customer_id, I.item_id
HAVING total_quantity > 0;
"""

cursor.execute(query)

# Fetch the results into a list of tuples
results = cursor.fetchall()

# Close the database connection
conn.close()

# Print or process the results as needed
for row in results:
    customer_id, item_id, total_quantity = row
    print(f"Customer {customer_id}: Item {item_id}, Total Quantity: {total_quantity}")


Customer 1: Item 1, Total Quantity: 10


# Using Numpy

In [21]:
import sqlite3
import numpy as np

# Connect to the SQLite database
conn = sqlite3.connect('sales_database.db')
cursor = conn.cursor()

# Define and execute the SQL query
query = """
SELECT
    C.customer_id,
    I.item_id,
    SUM(COALESCE(SD.quantity, 0)) AS total_quantity
FROM Customers C
JOIN Sales S ON C.customer_id = S.customer_id
JOIN SalesDetails SD ON S.transaction_id = SD.transaction_id
JOIN Items I ON SD.item_id = I.item_id
WHERE C.age BETWEEN 18 AND 35
GROUP BY C.customer_id, I.item_id
HAVING total_quantity > 0;
"""

cursor.execute(query)

# Fetch the results into a list of tuples
results = cursor.fetchall()

# Close the database connection
conn.close()

# Convert the results into a NumPy array
data = np.array(results)

# Create a mask for rows with total_quantity > 0
mask = data[:, 2] > 0

# Filter the rows with total_quantity > 0
filtered_data = data[mask]

# Print or process the filtered data as needed
for row in filtered_data:
    customer_id, item_id, total_quantity = row
    print(f"Customer {customer_id}: Item {item_id}, Total Quantity: {total_quantity}")


Customer 1: Item 1, Total Quantity: 10


# 3. store the query to a CSV file, delimiter should be the semicolon character (';')


In [22]:
import sqlite3
import csv

# Connect to the SQLite database
conn = sqlite3.connect('sales_database.db')
cursor = conn.cursor()

# Define and execute the SQL query
query = """
SELECT
    C.customer_id,
    I.item_id,
    SUM(COALESCE(SD.quantity, 0)) AS total_quantity
FROM Customers C
JOIN Sales S ON C.customer_id = S.customer_id
JOIN SalesDetails SD ON S.transaction_id = SD.transaction_id
JOIN Items I ON SD.item_id = I.item_id
WHERE C.age BETWEEN 18 AND 35
GROUP BY C.customer_id, I.item_id
HAVING total_quantity > 0;
"""

cursor.execute(query)

# Fetch the results into a list of tuples
results = cursor.fetchall()

# Close the database connection
conn.close()

# Define the output CSV file name
csv_filename = 'sales_results.csv'

# Write the results to a CSV file with semicolon delimiter
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=';')

    # Write the header
    csvwriter.writerow(['customer_id', 'item_id', 'total_quantity'])

    # Write the data rows
    for row in results:
        csvwriter.writerow(row)

print(f'Results have been saved to {csv_filename}')


Results have been saved to sales_results.csv
